# Transformer
- 구글이 발표한 [Attention is all you need](https://arxiv.org/pdf/1706.03762) 논문에서 제안된 모델입니다.
  - Attention은 17에서 다루었듯, Seq2Seq의 단점을 보완하기 위한 구조였는데, 해당 논문에서는 Attention만을 이용해서 제작한 모델을 소개합니다.

## Seq2Seq의 한계
- Seq2Seq은 Encoder를 이용하여 Sequence를 하나의 Context Vector로 압축하여 표현하고, Context Vector를 이용하여 Output Sequence를 제작합니다.
  - 이때, Encoder에서 하나의 벡터로 압축하며, 정보의 손실이 발생하기에 long sequence에서는 문제가 존재했습니다.
  - 이것을 보정하기 위해 Attention Mechanism이 도입되었습니다.
- 그렇다면, **Attention만으로 Encoder, Decoder를 구성하면 어떨까?** 이것이 Transformer의 시작입니다.


## Hyperparameter of Transformer
- $d_{model}$
  - Encoder, Decoder에서 정해진 차원이며, 임베딩 벡터의 차원입니다.
  - 각 Encoder, Decoder가 다음 Layer의 Encoder Decoder로 값을 보낼 때도 이 Dimension을 유지합니다.
  - 논문에서는 512를 사용했습니다.
- *num_layers*
  - 각 층은 Encoder와 Decoder로 이루어지는데, 이 층을 몇 개로 설정할지에 관한 파라미터입니다.
  - 논문에서는 6을 사용했습니다.
- *num_heads*
  - Transformer 논문에서는 Attetion을 한 번 하는 것이 아닌 병렬로 연결해서 결과를 합치는 방식으로 이용합니다.
  - num_heads는 이때, 병렬로 이용할 Attention의 개수입니다.
- $d_{ff}$
  - Transformer 내부의 FeedForwardNeuralNetwork의 크기를 나타냅니다.
  - 신경망의 입력층과 출력층의 크기는 위에서 언급하였던 $d_{model}$입니다.

## Structure of Transformer
- Seq2Seq
  - Encoder와 Decoder에서 각각 하나의 RNN이 t개의 시점을 가지고 동작합니다.
- Transformer
  - Encoder와 Decoder가 N개로 구성되어 동작합니다.

## Input of Transformer
- Transformer의 Input은 Seq2Seq과 다르게 각 단어의 임베딩을 직접 받는 것이 아닌, 조정된 임베딩 벡터를 Input으로 받습니다.
  - Transformer는 RNN과 다르게 순차적으로 입력을 받는 것이 아닌 병렬적으로 동작합니다.
  - 따라서, Transformer에서는 각 단어의 위치 관계를 다른 방식으로 전달해줄 필요가 있습니다.
  - 이때 사용하는 방법이 임베딩 벡터에 위치 정보를 더하여 전달해주는 것이고
    - 이것을 **Positional Encodeing**이라고 합니다.
### Positional Encoding
- Positional Encoding이 위치정보를 반영하게 해야하므로 논문에서는 아래와 같은 식을 사용하였습니다.
  - $PE_{pos,2i} = sin(\frac{pos}{{10000}^{ 2i /d_{model}}})$
  - $PE_{pos,2i+1} = cos(\frac{pos}{{10000}^{ 2i /d_{model}}})$
  - $pos$
    - 입력 문장 안에서 임베딩 벡터의 위치
  - $i$
    - 임베딩 벡터 내의 차원의 인덱스
  - $d_{model}$
    - 모든 층에서 입,출력의 차원
    - 임베딩 벡터도 $d_{model}$의 차원을 가집니다.

## Transformer's three attentions

### Encoder Self-Attention
- Encoder에서 이루어지는 Self-Attention

### Masked Decoder Self Attention
- Decoder에서 이루어지는 Self-Attention

### Encoder Decoder Attention
- Decoder에서 이루어지는 Attention
  - Self-Attention이 아닌 이유
    - Self-Attention은 Q,K,V가 모두 출처가 동일한 상황에서의 Attention을 말합니다.
    - 하지만, Encoder Decoder Attention에서는 Query가 Decoder의 벡터, Key,Value가 Encoder의 벡터이므로 Self-Attention이라고 하지 않습니다.


## Encoder
- Encoder의 구조는 아래와 같습니다.
  - <img src="https://blog.kakaocdn.net/dn/tGlEn/btru8JnUBPG/8ZXAURnTmIvkskTK6xTrrk/img.png" width="30%" height="30%"/>
  - 이러한 Encoder가 N개의 층으로 이루어집니다. ( 논문에서는 6개의 Layer로 이용했습니다. )
    - 한 Layer는 2개의 Sub-Layer로 이루어집니다. ( self attention, FFNN )

### Self-Attention
-  Attention
  - Query에 대해 모든 Key와의 유사도를 구합니다.
  - 해당 유사도를 가중치(Attention Weight)로 하여 Key와 매핑 되어있는 각 Value에 반영해준 후, 가중합하여 최종 output(Attention Value)을 Return합니다.
- Self-Attention
  - Attention을 **자기자신**에 대해 수행
    - 따라서 Q,K,V의 정의가 변합니다.
  - Attention의 Q,K,V
    - Q : **t 시점**에서의 **Decoder cell**에서의 hidden state
    - K : **모든 시점**에서의 **encoder cell**의 hidden states
    - V : **모든 시점**에서의 **encoder cell**의 hidden states
    - 하지만, t 시점은 1~n까지 모든 시점을 반복하기 때문에 결국 전체 시점으로 일반화가 가능합니다.
      - Q : **모든 시점**에서의 **Decoder cell**에서의 hidden state
      - K : **모든 시점**에서의 **encoder cell**의 hidden states
      - V : **모든 시점**에서의 **encoder cell**의 hidden states
  - Self-Attention의 Q,K,V
    - self-attention에서는 Q,K,V가 모두 동일합니다.
      - Q : 입력 문장의 모든 단어 벡터들
      - K : 입력 문장의 모든 단어 벡터들
      - V : 입력 문장의 모든 단어 벡터들
    - 이렇게 입력 문장의 모든 단어 벡터들에 대해 실행이 되면, 아래와 같이 단어간의 관계를 파악할 수 있습니다.
      - <img src="https://blog.kakaocdn.net/dn/bC21ZR/btrTAA08SDx/8FouR939D43iG1TRMGuxlK/img.png" width="30%" height="30%"/>
- Q,K,V Vector
  - self-attention에서 **입력 문장의 모든 단어 벡터들**을 이용하는데 여기서 단어 벡터는 $d_{model}$의 dimension을 가지는 단어 벡터들을 의미하는 것이 아닙니다.
    - Q,K,V는 $d_{model}$보다 작은 dimension을 가지는 단어 벡터들을 의미하고 논문에서는 아래와 같이 설정하였습니다.
      - $d_{model} = 512$ , $d_{Q,K,V} = 64$
      - 여기서 **64**라는 숫자는 $d_{model}$(512)을 num_heads(8)로 나눈 값입니다.
  - 각 Q,K,V vector들은 기존 단어 벡터로부터 Weighted Matrix를 곱하여 구할 수 있습니다.
    - 따라서, Weighted Matrix는 아래와 같은 크기를 같습니다.
      - $d_{model}$ x ($d_{model}$/num_heads)
- Scaled dot-product attention
  - Q,K,V 벡터를 얻은 이후엔 기존 Attention과 동일합니다.
    - 주어진 Query에 대해 모든 Key와의 가중치를 구한 후, 가중치를 이용하여 모든 V를 가중합하여 Attention Value를 구하는 것입니다.
  - Transformer에서는 Attention의 여러 종류 중, scaled dot-product attention을 이용합니다.
    - $score(q,k) = \frac{q⋅k}{\sqrt{n}} $ ($n$ = $d_k$(64))
  - 이것을 모든 q에 대해 반복하며 Attention Value를 구합니다.
- Attention Value 행렬 연산 처리
  - 모든 q에 대해 하나씩 반복하며 계산하면 시간이 많이 걸립니다. 따라서 행렬 연산을 이용하여 일괄로 처리합니다.
  - 과정
    1. 단어 별로 Q,K,V Vector를 구하는 것이 아니라, 입력 문장 행렬에 Weighted Matrix를 곱하여 Q,K,V Matrix를 구합니다.
    2. 구한 Matrix를 아래와 같이 연산합니다.
      - $Q$ x $K^T$ ( Q,K는 같은 Shape을 가지므로 Transpose를 해야 연산이 가능합니다. )
      - 위 식의 연산 결과로 생긴 행렬은 Attention Score Matrix가 됩니다.
        - Q의 모든 단어, K의 모든 단어들에 대해 곱했으므로, Matrix의 모든 원소는 내적 값을 갖게 됩니다.
        - 따라서 Matrix 전체에 $\sqrt{n}$을 나눠주면 Scaled dot-product attention score matrix가 됩니다.
    3. 구한 Attention Score Matrix에 Softmax를 사용하여 distribution을 구합니다.
    4. distribution (weight)를 모든 V에 적용하여 Weighted sum을 구하면 Attention Value Matrix가 결과로 나옵니다.
  - 이것을 수식으로 정리하면 아래와 같습니다.
    - $Attention(Q,K,V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$
- Multi-head attention
  - 위의 Self-Attention에서 $d_{model}$의 차원을 가진 단어 벡터를 그대로 이용하지 않고, num_heads로 나눈 $d_{Q,K,V}$차원을 가진 벡터를 이용하였습니다.
  - 여기에서 num_heads의 의미와 $d_{model}$이 아닌 축소된 차원을 가진 벡터로 수행하였는지 설명합니다.
  - Attention head
    - num_heads 개수만큼 생성되는 attention value matrix
    - 이렇게 여러 개의 Attention value matrix를 활용하는 것은 여러 시각에서 정보들을 수집하겠다는 의미입니다.
  - 여러 개의 Attention value matrix를 얻었으면, 이것을 concatenate합니다.
    - concatenate하게 되면 차원이 아래처럼 변합니다.
      - 기존 : (seq_len,$d_q$)
      - 변경 후 : (seq_len,$d_{model}$)
  - Multi-head Attention Matrix
    - 위에서 concatenate한 matrix에 새로운 Weighted Matrix ($W^o$)를 곱해서 Multi-head attention matrix를 구합니다.
    - 이 때 차원은 그대로 유지됩니다. (seq_len,${d_{model}}$)
  - 결론적으로, Input sequence (seq_len,${d_{model}}$)을 multi-head attention 처리를 해도 동일하게 차원이 유지되는 것입니다.
    - 이렇게 동일하게 유지함으로서, 여러 개의 encoder layer를 거칠 수 있습니다.
- Padding Mask
  - 입력 문장에 <PAD> token이 존재하는 경우 Attention에서 제외하기 위한 부분입니다.
    - <PAD>는 길이를 맞추기 위해 들어간 토큰이므로 실질적 의미를 가지지 않습니다.
    - 따라서, 이 부분은 Attention 연산에 포함되지 않게 Masking을 합니다.
  - 방식
    - <PAD>가 있는 열에 -inf를 넣어줍니다.
      - -inf를 넣으면, softmax function을 지난 이후에는 해당 값이 0이 됩니다.
### Position-wise FFNN
- Fully-connected FFNN
  - $FFNN(x) = MAX(0,xW_1 + b_1)W_2 + b_2$
    - MAX(0,x) -> ReLU를 의미합니다.
    - 따라서, 과정을 정리하면 아래와 같습니다.
      1. input x에 대해 W_1을 곱하고, b_1을 더함
      2. 1의 결과를 ReLU 활성화 함수를 거침
      3. 2의 결과를 W_2를 곱하고, b_2를 더함
    - W_1.shape == ($d_{model}(512),d_{ff}(2048)$)
    - W_2.shape == ($d_{ff}(2048),d_{model}(512)$)
### Residual Connection & Layer Normalization
- Residual Connection
  - Sub Layer의 input, output을 더하는 것
   - $x + sublayer(x)$
  - [Deep Residual Learning for Image Recognition 논문](https://arxiv.org/pdf/1512.03385)
- Layer Normalization
  - Residual Connection을 거친 결과는 Layer Normalization을 진행합니다.
    - $LN = LayerNorm(x+sublayer(x))$
  - Layer Normalization은 Tensor's last dimension을 이용하여 average와 variance를 구합니다.
    - 여기서 last dimension 은 (seq_len,$d_{model}$)이므로 $d_{model}$이 됩니다.
    - (행,열)이므로 last dimension에 대해 average와 variance를 구한다는 것은 열에 대한 average,variance를 구하는 것입니다. (행 방향 -> )
  - layer normalization을 수행한 후에는 $x_i$는 $ln_i$가 됩니다.
  - 수식
    - $\hat{x}_{i,k} = \frac{x_{i,k}-μ_i}{\sqrt{σ^2 + ϵ}}$
      - ϵ은 분모가 0이 되는 것을 방지합니다.
    - 위의 수식으로 $\hat{x}_{i,k}$를 구할 수 있고, 여기에 γ와 β (learnable parameter)를 추가하여 최종 Layer Normalization 수식을 얻을 수 있습니다.
      - $ln_i= γ\hat{x}_i+β = LayerNorm(x_i)$
      - [Layer Normalization 논문](https://arxiv.org/pdf/1607.06450)

### From Encoder To Decoder
- Encoder에서 Multi-head self-attetion ~ Add&LayerNorm 까지를 num_layers 만큼 반복을 하고 난 후, decoder로 결과를 전달합니다.

### Decoder's First sub-layer : self-attetion, look-ahead mask
- Decoder도 Encoder와 동일한 구조로 이루어져 있습니다. 구조는 아래와 같습니다.
  - ![Decoder](https://upload.wikimedia.org/wikipedia/commons/5/55/Transformer%2C_one_decoder_block.png)
- Transformer 또한 Seq2Seq과 같이 Teacher Forcing 방식으로 학습을 진행합니다.
  - Teacher Forcing
    - 정답을 미리 제공해주며 학습하는 방식
  - 그러나, RNN기반인 Seq2Seq은 각 시점마다 1개씩 가져오는 것과 다르게 Transformer는 전체 정답을 한 번에 입력받습니다.
    - 이것은 1,2,3,4 순서의 token이 있을 때, 3,4를 보고 2를 예측할 수도 있다는 문제가 존재합니다.
    - 따라서, look-ahead mask를 도입하여 현재 시점 이후의 단어를 참고하지 못하도록합니다.
- Look-ahead Mask
  - Decoder's first sub-layer (Masked Multi-head self-attention)
    - Encoder's self-attention과 동일한데, Mask가 추가되었다는 점에서만 차이가 존재합니다.
    - 따라서 Masked Multi-head self-attention 도 <PAD> mask는 동일하게 존재하고, 거기에 추가로 look-ahead Mask가 추가된 것입니다.
- Encoder-Decoder Attention
  - Decoder's second sub-layer ( self-attention 이 아닙니다. )
  - 여기에서는 Encoder-Decoder Attention으로 Query가 Decoder Matrix, Key,Value가 Encoder Matrix입니다.
  - Query가 Decoder Matrix이고, Key가 Encoder Matrix이므로 Attention Score는
    - 입력 문장과 결과 문장의 관계를 나타냅니다.

- 여기까지가 Transformer 정리입니다.
  - [Transformer Details Not Described in The Paper](https://tunz.kr/post/4)
